# Preprocessing

In [1]:
import os
import json
import ast
import pandas as pd
from pprint import pprint
from collections import Counter
import os.path as path
from pathlib import Path



def binarize(row):
    if row!='Pro':
        return 0
    else:
        return 1

def save_as_json(a_dict, path):
    with open(path, 'w') as fp:
        json.dump(a_dict, fp, indent=2)
    
    
def save_as_txt(file_path, my_list):
    if not os.path.exists(os.path.dirname(file_path)):
        os.mkdir(os.path.dirname(file_path))  
    with open(file_path, "w") as my_file:
        my_file.write(my_list)


root = path.abspath(path.join(os.getcwd(),"../.."))

df = pd.read_csv('/Users/user/differential-bias_3/data/raw/Webis-argument-framing.csv')

df['stance']=df['stance'].str.replace(' ', '')
df['stance_id'] = df['stance'].apply(binarize)
df

,argument_id,frame_id,frame,topic_id,topic,premise,stance,conclusion,stance_id
0,0,0,economics,0,$700 billion US economic bailout,U.S. Treasury Secretary Henry Paulson summariz...,Pro,Not passing $700b bailout risks sending econom...,1
1,1,0,economics,0,$700 billion US economic bailout,"""Job security: Safeguarding jobs across the e...",Pro,$700b bailout helps avoid widespread bankruptc...,1
2,2,0,economics,0,$700 billion US economic bailout,Treasury Secretary Henry Paulson summarized th...,Pro,$700b bailout is generally well designed to so...,1
3,3,0,economics,0,$700 billion US economic bailout,"""Paulson Plan provides a plan Just as optimis...",Pro,$700b bailout offers buyer for frozen mortgage...,1
4,4,0,economics,0,$700 billion US economic bailout,"""The Treasury proposal to rescue the financia...",Pro,Most economists support the $700b US economic ...,1
...,...,...,...,...,...,...,...,...,...
12321,14110,1622,endangered,1823,Zoos,While some argue that zoos are a means to prot...,Pro,Most of the animals that you see in zoos aren'...,1
12322,14111,1622,endangered,1823,Zoos,According to the World Conservation Union whic...,Pro,Zoos are not capable of sustaining all endange...,1
12323,14112,1622,endangered,1823,Zoos,Zoos are a good place to house endangered spec...,Con,Zoos help protect endangered species.,0
12324,14113,1622,endangered,1823,Zoos,If natural or human factors have made a specie...,Con,Zoos help breed endangered animals.,0


In [2]:
output_dir = root+'/data/processed/bite-the-bytes-20/'
print('-> Folder structure created!', output_dir)

Path(output_dir).mkdir(parents=True, exist_ok=True)

for x in df.itertuples():
    save_as_txt(str('{}/argunits-topic{}/topic{}-unit{}-frame{}-stance{}.txt'.format(output_dir, f"{x[4]:0>4}", f"{x[4]:0>4}",f"{x[1]:0>5}", f"{x[2]:0>4}", f"{x[9]}")), 'Premise: '+x[6]+'\n\nConclusion: '+x[8])    
    save_as_txt(str('{}/argunits-topic{}/topic{}.txt'.format(output_dir, f"{x[4]:0>4}", f"{x[4]:0>4}")), x[5])
    save_as_txt(str('{}/argunits-topic{}/frame{}.txt'.format(output_dir, f"{x[4]:0>4}", f"{x[2]:0>4}")), x[3])
    

-> Folder structure created! /Users/user/bias_reorganizated/data/processed/bite-the-bytes-20/


In [3]:
# Create frame mapping: id:frame (e.g., 0000 : economics)
my_dict= dict(zip(df['frame_id'].values.tolist(), df['frame'].values.tolist()))
list_of_strings = [ f'{key:0>4} : {my_dict[key]}' for key in my_dict ]

#pprint(my_dict)

#Save into a .txt file
with open(output_dir+'/frames.txt', 'w') as my_file:
    [my_file.write(f'{st}\n') for st in list_of_strings]

print('-> Frame ID file created!')

-> Frame ID file created!


## Frame/Argument distribution summary

In [4]:
arg_num = df['argument_id'].values.tolist()
frame_num = df['frame_id'].values.tolist()
    
d = []
keys=[]
values = []
    
for e in arg_num:
    temp_df = df[df['topic_id']==e]

    try:
        topic = temp_df['topic'].tolist()
        pro = temp_df[temp_df['stance']=='Pro'].shape[0]
        con = temp_df[temp_df['stance']=='Con'].shape[0]
        size = len(temp_df['topic_id'].to_list())
        frames = dict(Counter(df[df['topic_id']==e]['frame'].tolist()))
        frames_id = dict(Counter(df[df['topic_id']==e]['frame_id'].tolist()))
        stance_dict = {'pro':pro,'con':con}
        topic_type = ' '.join(str(topic[0]).split())
        
    except IndexError:
        topic_type = 'null'

    #pprint(ast.literal_eval("{{'topic':{}, 'size':{}, 'frames':{}, 'stance':{}}}".format(e,size,frames,stance_dict)))
    #pprint(ast.literal_eval("{{'topic_id':{}, 'topic_type':'{}', 'size':{}, 'frames':{}, 'stance':{}}}".format(e,topic_type,size,frames,stance_dict)))
    d.append(ast.literal_eval("{{'topic_id':{}, 'topic_type':'{}', 'size':{}, 'frames':{}, 'stance':{}}}".format(e,topic_type,size,frames,stance_dict)))


In [5]:
l = [e for e in d if e['topic_type'] != 'null']
#l

In [6]:
frame_arg_distro = root+'/data/processed/bite-the-bytes-20/frame_argument_distro.json'

with open(frame_arg_distro, 'w') as fout:
    json.dump(l, fout, indent=4)

In [10]:
with open(frame_arg_distro) as json_file:
    data = json.load(json_file)
    
data

[{'topic_id': 0,
  'topic_type': '$700 billion US economic bailout',
  'size': 46,
  'frames': {'economics': 11,
   'immediacy': 4,
   'intervention': 6,
   'moral hazard': 2,
   'wallstreet': 6,
   'taxpayers': 6,
   'public opinion': 5,
   'powers': 4,
   'alternatives': 2},
  'stance': {'pro': 23, 'con': 23}},
 {'topic_id': 8,
  'topic_type': '2009 US economic stimulus',
  'size': 47,
  'frames': {'government stimulus': 8,
   'consumer confidence': 2,
   'size': 4,
   'growing government': 6,
   'spending vs. tax cuts': 3,
   'immediate stimulus': 4,
   'good/bad stimulus': 3,
   'infrastructure': 2,
   'consequences': 5,
   'debt': 4,
   'buy american': 1,
   'public opinion': 3,
   'bi-partisanship': 2},
  'stance': {'pro': 28, 'con': 19}},
 {'topic_id': 10,
  'topic_type': '2010 US bank tax',
  'size': 21,
  'frames': {'taxpayer money': 9,
   'deficit reduction': 3,
   'stability': 4,
   'jobs': 2,
   'vs other bailouts': 3},
  'stance': {'pro': 10, 'con': 11}},
 {'topic_id': 12,

In [11]:
# Filter
list(filter(lambda d: 0 == d['topic_id'], data))

[{'topic_id': 0,
  'topic_type': '$700 billion US economic bailout',
  'size': 46,
  'frames': {'economics': 11,
   'immediacy': 4,
   'intervention': 6,
   'moral hazard': 2,
   'wallstreet': 6,
   'taxpayers': 6,
   'public opinion': 5,
   'powers': 4,
   'alternatives': 2},
  'stance': {'pro': 23, 'con': 23}}]

## Frame Stance Distribution

In [12]:

# Helpers

def get_frame_info(temp_df):
    frames = temp_df.groupby(['frame','stance'])
    frame_info = dict(frames.describe()['frame_id']['count'].astype(int))
    return frame_info

def reformat_dict(frame_dict):
    out_dict = {}
    for k,v in frame_dict.items():
        frames,PC = k
        if frames not in out_dict:
            out_dict[frames] = {'stance': {}}
        out_dict[frames]['stance'][PC] = v
    return out_dict

###########################################


def get_frame_stance_summary(df):
    d = []
    a_dict = {}

    arg_num = df['argument_id'].values.tolist()
    frame_num = df['frame_id'].values.tolist()
    
    for e in arg_num:
        temp_df = df[df['topic_id']==e]
        try:
            topic = temp_df['topic'].tolist()
            pro = temp_df[temp_df['stance']=='Pro'].shape[0]
            con = temp_df[temp_df['stance']=='Con'].shape[0]
            size = len(temp_df['topic_id'].to_list())
            topic_id = temp_df['topic_id'].to_list()[0]
    
            frames_id = dict(Counter(df[df['topic_id']==e]['frame_id'].tolist()))        
            topic_type = ' '.join(str(topic[0]).split())
            frame_info = reformat_dict(get_frame_info(temp_df))
            
            a_dict["topic"] = topic_type
            a_dict["topic_id"] = topic_id      
            a_dict["frame_info"] = frame_info
            a_dict["frame_info"]["total_pro"] = pro
            a_dict["frame_info"]["total_con"] = con
            a_dict["frame_info"]["num_of_stances"] = con+pro
            a_dict["frame_info"]["frames_id"] = list(frames_id.values())
        
            with open('temp_out.txt', 'a') as f:
                print(a_dict, file=f)
            
        except IndexError:
            pass
    
    with open('temp_out.txt') as f:
        for line in f:
            d.append(ast.literal_eval(line))
    os.remove("temp_out.txt")
    
    with open(root+'/data/processed/frame_stance_summary.json', 'w') as fout:
        json.dump(d, fout, indent=4)

get_frame_stance_summary(df)

In [13]:
pd.read_json(root+'/data/processed/frame_stance_summary.json')

,topic,topic_id,frame_info
0,$700 billion US economic bailout,0,"{'alternatives': {'stance': {'Con': 1, 'Pro': ..."
1,2009 US economic stimulus,8,"{'bi-partisanship': {'stance': {'Pro': 2}}, 'b..."
2,2010 US bank tax,10,"{'deficit reduction': {'stance': {'Con': 2, 'P..."
3,4-team college football playoff,12,"{'champs': {'stance': {'Con': 1, 'Pro': 1}}, '..."
4,700 mile US Mexico border fence,13,"{'border security': {'stance': {'Con': 9, 'Pro..."
...,...,...,...
435,Women in combat,1805,"{'capabilities': {'stance': {'Con': 1, 'Pro': ..."
436,Year-round school,1816,"{'achievement': {'stance': {'Con': 2, 'Pro': 3..."
437,Yucca Mountain nuclear waste repository,1820,{'fossil fuel dependence': {'stance': {'Pro': ...
438,Zero tolerance law,1822,"{'costs and economics': {'stance': {'Con': 1, ..."
